In [1]:
import pandas as pd
import numpy as np
from sklearn.externals import joblib
import nltk
import gensim
from tqdm import tqdm_notebook

from sklearn import metrics

import torch as tt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchtext.data import Field, LabelField, BucketIterator, ReversibleField, TabularDataset

from string import punctuation
from nltk.corpus import stopwords
import warnings
from tqdm import tqdm_notebook

warnings.filterwarnings('ignore')

SEED = 42
np.random.seed(SEED)

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [3]:
stops = set(stopwords.words('english'))
punct = punctuation+'«»—…“”*№–'

import spacy


spacy_en = spacy.load('en')
spacy_en.remove_pipe('tagger')
spacy_en.remove_pipe('ner')


def tokenizer1(text):
    return [tok.lemma_ for tok in spacy_en.tokenizer(text)]
    #words = [word for word in text.lower().split()]
    #return words

def tokenizer2(text):
    words = [word.strip(punct) for word in text.lower().split()]
    return words

In [4]:
class MyModel(nn.Module):
    
    def __init__(self, vocab_size, embed_size, hidden_size, w2v=False, drop=False, mod='text'):
        super(MyModel, self).__init__()
        
        if w2v is True:
            global weights 
            self.embedding = nn.Embedding.from_pretrained(weights, freeze=True)
        else:
            self.embedding = nn.Embedding(vocab_size, embed_size)
           # tt.nn.init.uniform_(self.embedding.weight)
        
        self.mod = mod
        
        if drop is True:
            self.drop_en = nn.Dropout(p=0.6)
        else:
            self.drop_en = False
        
        self.rnn = nn.LSTM(input_size=embed_size,
                           hidden_size=hidden_size,
                           bidirectional=True,
                           batch_first=True,
                          )
        self.fc = nn.Linear(hidden_size * 2 *2, 2)
        
    def forward(self, batch):
        
        if self.mod == 'text':
            x, x_lengths = batch.text
        if self.mod == 'comment':
            x, x_lengths = batch.comment
        if self.mod == 'parent_comment':
            x, x_lengths = batch.parent_comment
        
        x = self.embedding(x)
        if self.drop_en is not False:
            x = self.drop_en(x)

        if x_lengths is not None:
            x_lengths = x_lengths.view(-1).tolist()
            x = nn.utils.rnn.pack_padded_sequence(x, x_lengths, batch_first=True)
            
        _, (hidden, cell) = self.rnn(x)
        
        hidden = hidden.transpose(0,1)
        cell = cell.transpose(0,1)
        hidden = hidden.contiguous().view(hidden.size(0),-1)
        cell = cell.contiguous().view(cell.size(0),-1)
        x = tt.cat([hidden, cell], dim=1).squeeze(1)
        x = self.fc(x)
        return x

In [5]:
def _train_epoch(model, iterator, optimizer, criterion, curr_epoch):

    model.train()

    running_loss = 0

    n_batches = len(iterator)
    iterator = tqdm_notebook(iterator, total=n_batches, desc='epoch %d' % (curr_epoch), leave=True)

    for i, batch in enumerate(iterator):
        optimizer.zero_grad()

        pred = model(batch)
        loss = criterion(pred, batch.label)
        loss.backward()
        optimizer.step()

        curr_loss = loss.data.cpu().detach().item()
        
        loss_smoothing = i / (i+1)
        running_loss = loss_smoothing * running_loss + (1 - loss_smoothing) * curr_loss

        iterator.set_postfix(loss='%.5f' % running_loss)

    return running_loss

def _test_epoch(model, iterator, criterion):
    model.eval()
    epoch_loss = 0

    n_batches = len(iterator)
    with tt.no_grad():
        for batch in iterator:
            pred = model(batch)
            loss = criterion(pred, batch.label)
            epoch_loss += loss.data.item()

    return epoch_loss / n_batches


def nn_train(model, train_iterator, valid_iterator, criterion, optimizer, n_epochs=100,
          scheduler=None, early_stopping=0, cri=False):

    prev_loss = 100500
    es_epochs = 0
    best_epoch = None
    history = pd.DataFrame()

    for epoch in range(n_epochs):
        train_loss = _train_epoch(model, train_iterator, optimizer, criterion, epoch)
        valid_loss = _test_epoch(model, valid_iterator, criterion)

        valid_loss = valid_loss
        print('validation loss %.5f' % valid_loss)

        record = {'epoch': epoch, 'train_loss': train_loss, 'valid_loss': valid_loss}
        history = history.append(record, ignore_index=True)

        if early_stopping > 0:
            if valid_loss > prev_loss:
                es_epochs += 1
            else:
                es_epochs = 0

            if es_epochs >= early_stopping:
                best_epoch = history[history.valid_loss == history.valid_loss.min()].iloc[0]
                print('Early stopping! best epoch: %d val %.5f' % (best_epoch['epoch'], best_epoch['valid_loss']))
                break

            prev_loss = min(prev_loss, valid_loss)
        
        if cri is True:
            scheduler.step(valid_loss)

In [6]:
def predict(batch, model, proba=True):
    
    global TEXT
    prediction = tt.softmax(model.forward(batch), dim=-1)
    prediction = prediction.detach().numpy()
    
    if proba is True: return prediction
    else: return prediction.argmax(axis=1)

In [7]:
def text_formaion(path, tok='tokenizer1', field='comment', max_size=30000):
    
    if tok == 'tokenizer1':
        TEXT = Field(include_lengths=True, batch_first=True, 
                 tokenize=tokenizer1,
                 eos_token='<eos>',
                 lower=False,
                 stop_words=nltk.corpus.stopwords.words('english')
                )
    else:
        TEXT = Field(include_lengths=True, batch_first=True, 
                 tokenize=tokenizer2,
                 eos_token='<eos>',
                 lower=False,
                 stop_words=nltk.corpus.stopwords.words('english')
                )

    LABEL = LabelField(dtype=tt.int64, use_vocab=False)

    dataset = TabularDataset(path, format='csv', 
                         fields=[('label', LABEL), (field, TEXT)], 
                         skip_header=True)
    
    TEXT.build_vocab(dataset, min_freq=5, max_size=max_size)
    print(len(TEXT.vocab.itos))
    LABEL.build_vocab(dataset)
    train, test = dataset.split(0.8, stratified=True)
    train, valid = train.split(0.7, stratified=True)
    
    return TEXT, train, test, valid

In [8]:
def model_train(TEXT, train, valid, test, batch_size=100, w2v=False, drop=False, mod='text', n_epochs=10, embed_size=100, cri=False):
    
    batch_size = batch_size 

    model = MyModel(len(TEXT.vocab.itos),
                embed_size=embed_size,
                hidden_size=128,
                w2v=w2v, drop=drop, mod=mod)

    if mod == 'text':
        train_iterator, valid_iterator, test_iterator = BucketIterator.splits((train, valid, test),
        batch_sizes=(batch_size, batch_size, batch_size), shuffle=True, sort_key=lambda x: len(x.text),
        sort_within_batch=True,)
    if mod == 'comment':
        train_iterator, valid_iterator, test_iterator = BucketIterator.splits((train, valid, test),
        batch_sizes=(batch_size, batch_size, batch_size), shuffle=True, sort_key=lambda x: len(x.comment),
        sort_within_batch=True,)
    if mod == 'parent_comment':
        train_iterator, valid_iterator, test_iterator = BucketIterator.splits((train, valid, test),
        batch_sizes=(batch_size, batch_size, batch_size), shuffle=True, sort_key=lambda x: len(x.parent_comment),
        sort_within_batch=True,)
        

    optimizer = optim.Adam(model.parameters())
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=7)
    criterion = nn.CrossEntropyLoss()
    
    nn_train(model, train_iterator, valid_iterator, criterion, optimizer, scheduler=scheduler, 
        n_epochs=n_epochs, early_stopping=2, cri=cri)
    
    return model, test_iterator

In [9]:
def acc_score(model, test_iterator, proba=False):
    res = []
    t_par = tqdm_notebook(test_iterator, desc='i', leave=True)

    for i in t_par:
        pred = predict(i, model, proba=proba)
        res.append(accuracy_score(np.array(i.label), pred))

    return np.mean(res)

In [61]:
w2v_model = gensim.models.KeyedVectors.load_word2vec_format('../input/googlenewsvectorsnegative300/GoogleNews-vectors-negative300.bin', binary=True)

weights = tt.FloatTensor(w2v_model.vectors)

In [62]:
path = '../input/my-sarcasm/train-balanced-sarcasm2.csv'

Колонка - comment, с удалением пунктуации, сохранение капса, первоначальный код

In [ ]:
TEXT2, train2, test2, valid2 = text_formaion(path, tok='tokenizer2', field='comment', max_size=30000)
model2, test_iterator2 = model_train(TEXT2, train2, valid2, test2, batch_size=100, w2v=False, drop=False, mod='comment', n_epochs=10, embed_size=100, cri=False)
acc_score(model2, test_iterator2, proba=False)

Колонка - comment, без удаления пунктуации, сохранение капса, первоначальный код

In [65]:
TEXT, train, test, valid = text_formaion(path, tok='tokenizer1', field='comment', max_size=30000)
model, test_iterator = model_train(TEXT, train, valid, test, batch_size=100, w2v=False, drop=False, mod='comment', n_epochs=10, embed_size=100, cri=False)
acc_score(model, test_iterator, proba=False)

30003


validation loss 0.55293


validation loss 0.54385


validation loss 0.55331


validation loss 0.58325
Early stopping! best epoch: 1 val 0.54385


0.7127056919341785